# オペレーティングシステム 演習 5
# 仮想記憶

**このセルを編集して, 名前と学生証番号を書け.**

 * 名前 Name : 西山　晃人
 * 学生証番号 Student ID : 03-190503

**書けたら Shift + Enter で実行(入力を確定)させ, 保存(`Ctrl-S`)せよ**

## 1. はじめに

オペレーティングシステムが提供している「仮想記憶」と呼ばれるメモリ管理方式について, 概念だけでなく実感として理解するために,以下の簡単な演習を行う. 理解してほしい概念は以下のとおり.

* OSからメモリを(Unixであればsbrk, mmapなどのシステムコールを呼び出すことによって, またはmallocなどのライブラリ関数がそれらを結果的に呼び出すことによって)割り当ててもらったとする. 
* Mバイトのメモリを要求してアドレスAが返ってきたとする. それはアドレスA〜A+M-1 のM個の番地を使う(読み書きする)「権利」(比喩: 小切手)を得たということであり(<font color="blue">仮想メモリ</font>の割当て), 必ずしもその時点でそれらに対応する物理メモリ(比喩: 現金)が確保されているわけではない. というよりもほとんどの場合は確保されていない.
* 実際の物理メモリの割当はある大きさの連続したアドレスの固まり(<font color="blue">ページ</font>)を単位として行われる. 1ページは典型的には4096バイト, つまり2進数で下位12ビットだけが異なるアドレスの集合である
* あるページを割り当ててから初めてそのページのどこかにアクセスしたときに, <font color="blue">ページフォルト</font>が発生し, それを受けたOSがそのページに対する物理メモリ(物理ページ)を割り当てる(<font color="blue">要求時ページング; demand paging</font>). 以降で述べるメジャーページフォルトと区別してこちらを<font color="blue">マイナー(ページ)フォルト</font>と呼ぶ.

```
a = mmap(...);  // この時点では論理アドレス(ページ)だけが確保される
a[0] = 1;       // ここでページフォルト -> 物理アドレスが確保される
```

* 一旦確保された物理ページは, 物理メモリが足りなくならない限り開放されることはない. つまりある(論理)ページへの1度目のアクセスだけがページフォルトを起こし, 以降は起こさない, という理解でだいたいの場合正しいのだが, 他のプロセスで大量の物理メモリを使うなどして物理メモリが足りなくなると, 一旦確保された物理ページが開放される
* その際, その物理ページの中身は2次記憶(ハードディスクやSSD)に追い出される(<font color="blue">ページアウト; page out</font>.  なぜかLinuxでは<font color="blue">スワップアウト; swap out</font>と呼んでいる)
* その後もう一度その論理ページにアクセスすると再びページフォルトが発生し, 対応する2次記憶領域から中身が読み込まれる(<font color="blue">ページイン; page in</font>, Linuxでは<font color="blue">スワップイン; swap in</font>). このようなページフォルト, つまり2次記憶からの読み込みが必要なページフォルトを<font color="blue">メジャー(ページ)フォルト</font>と呼ぶ.
* ページフォルトが起きていようといまいとユーザプロセスから見れば, 確保されたアドレスは何事もなくアクセスできるので, このことを普段から意識する必要はないが, 同じメモリアクセスでもそのコストが全く違うことに注意が必要である

* 同じことを別の角度から説明する. ある時点での1つの論理ページの状態は以下に分類される.
 1. まだ一度も使われたことがなく, 物理メモリもディスクも消費していない
 1. 物理メモリ上に存在する
 1. 物理メモリ上に存在せず, ディスク上に存在する

あるページは, 割り当てられた当初は1. の状態で, 最初に使われた(loadもしくはsotre命令によって触られた)時に2. の状態になる. それ以降, 物理メモリの枯渇により2次記憶に追い出されると3.の状態になる. 以降はまたそのページにアクセスすると2次記憶からページが読み込まれ, 2.の状態になる. その後また追い出されると3. の状態, ... という具合.

## 2. 搭載物理メモリ量の確かめ方

いくつかの方法がある.

なお以下は通常のLinuxコマンドに過ぎないので自分のLinuxのコマンド端末に入力しても同じか, 似たような結果が得られるので試してみよ

**1. freeコマンド**

文字通り空き(free)メモリ量を表示するが, 同時に搭載メモリ量, swap (ページング)領域の大きさも表示する.

以下は搭載物理メモリ量が 32563000 KB (約32 GB), swap (ページング)領域の大きさが2097148 KB (約2 GB)ということ. そのうち使われているのが 3398120 (約3 GB).

```
              total        used        free      shared  buff/cache   available
Mem:       32563000     3398120    20188032      544868     8976848    28137604
Swap:       2097148           0     2097148
```

In [1]:
//% cmd: free

              total        used        free      shared  buff/cache   available
Mem:       65151292    18525536    34222188        2492    12403568    45905452
Swap:      16777212      990720    15786492


LinuxやMacな人は手元の端末でも試してみよ(以下いちいち断らない)

```
$ free
```

**2. cat /proc/meminfo**

Linuxは様々な情報を, 特別なファイルを経由して提供する. /proc/meminfo というファイルはメモリに関する情報を提供する.

以下に抜粋した3つは搭載物理メモリ量, 空きメモリ量, swap領域の量.

```
MemTotal:       32563000 kB
MemFree:        20024332 kB
   ...
SwapTotal:       2097148 kB
   ...
```

In [2]:
//% cmd: cat /proc/meminfo

MemTotal:       65151292 kB
MemFree:        34120360 kB
MemAvailable:   45809260 kB
Buffers:          718380 kB
Cached:         10955948 kB
SwapCached:       200860 kB
Active:         27205664 kB
Inactive:        1936800 kB
Active(anon):   17038540 kB
Inactive(anon):   432080 kB
Active(file):   10167124 kB
Inactive(file):  1504720 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:      16777212 kB
SwapFree:       15780348 kB
Dirty:               304 kB
Writeback:             0 kB
AnonPages:      17384456 kB
Mapped:           262892 kB
Shmem:              2492 kB
Slab:            1348888 kB
SReclaimable:     734876 kB
SUnreclaim:       614012 kB
KernelStack:       33280 kB
PageTables:       270804 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    49352856 kB
Committed_AS:   27702676 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
HardwareCorrupted:     0 kB
AnonHugePages:   

**3. top -b -n 1**

top は普通, 走っているプロセスを表示するために使うが, 先頭に搭載メモリ量も表示する.

```
KiB Mem : 32563000 total, 20286364 free,  3302428 used,  8974208 buff/cache
KiB Swap:  2097148 total,  2097148 free,        0 used. 28233964 avail Mem 
```

In [4]:
//% cmd: top -b -n 1

top - 09:30:39 up 28 days,  5:21,  0 users,  load average: 1.27, 0.63, 0.48
Tasks: 1310 total,   3 running, 1220 sleeping,   0 stopped,   0 zombie
%Cpu(s):  8.6 us,  1.5 sy,  0.0 ni, 89.9 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 65151292 total, 33822932 free, 18907700 used, 12420660 buff/cache
KiB Swap: 16777212 total, 15112956 free,  1664256 used. 45523372 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND
27147 u19300    20   0  697600  15428  14288 R 100.0  0.0   0:54.56 python3
26114 u19303    20   0  697604  57472  14296 R  94.7  0.1   0:01.38 python3
27608 u19303    20   0   35168   5172   3324 R  21.1  0.0   0:00.07 top
27603 u19300    20   0  311716 247724   1204 D  15.8  0.4   0:00.24 record_pag+
    1 root      20   0  228116  11984   6872 S   0.0  0.0   0:42.41 systemd
    2 root      20   0       0      0      0 S   0.0  0.0   0:00.32 kthreadd
    4 root       0 -20       0      0      0 I   0.0  0.0   0:00.00 kworker/0:+
    6 r

 3354 root      20   0  122692   7732   6756 S   0.0  0.0   0:00.00 sudo
 3358 root      20   0  165544   9616   8132 S   0.0  0.0   0:00.01 su
 3359 u19225    20   0   31176  10532   5800 S   0.0  0.0   0:00.04 echo_pid_a+
 3387 u19225    20   0    4628    888    820 S   0.0  0.0   0:00.00 sh
 3395 u19225    20   0  376812  85988  19876 S   0.0  0.1   0:03.11 jupyter-no+
 3404 root      20   0  122692   7792   6820 S   0.0  0.0   0:00.01 sudo
 3406 root      20   0  165544   9508   8028 S   0.0  0.0   0:00.00 su
 3407 u19226    20   0   31176  10580   5848 S   0.0  0.0   0:00.04 echo_pid_a+
 3422 u19226    20   0    4628    884    816 S   0.0  0.0   0:00.00 sh
 3436 u19226    20   0  378940  88240  19692 S   0.0  0.1   0:09.63 jupyter-no+
 3441 root      20   0  122692   7796   6820 S   0.0  0.0   0:00.00 sudo
 3442 root      20   0  165544   9540   8056 S   0.0  0.0   0:00.01 su
 3443 u19227    20   0   31176  10704   5976 S   0.0  0.0   0:00.04 echo_pid_a+
 3463 u19227    20   0    

 4980 u19257    20   0  376884  85500  19516 S   0.0  0.1   0:03.09 jupyter-no+
 4987 root      20   0  122692   7836   6860 S   0.0  0.0   0:00.01 sudo
 4988 root      20   0  165544   9692   8208 S   0.0  0.0   0:00.00 su
 4989 u19258    20   0   31176  10640   5912 S   0.0  0.0   0:00.04 echo_pid_a+
 5011 u19258    20   0    4628    812    744 S   0.0  0.0   0:00.00 sh
 5022 u19258    20   0  222380  78432  17644 S   0.0  0.1   0:02.43 jupyter-no+
 5027 root      20   0  122692   8016   7040 S   0.0  0.0   0:00.01 sudo
 5028 root      20   0  165544   9664   8184 S   0.0  0.0   0:00.01 su
 5029 u19259    20   0   31176  10588   5856 S   0.0  0.0   0:00.04 echo_pid_a+
 5044 u19259    20   0    4628    888    820 S   0.0  0.0   0:00.00 sh
 5059 u19259    20   0  222364  78428  17652 S   0.0  0.1   0:02.35 jupyter-no+
 5068 root      20   0  122692   7824   6848 S   0.0  0.0   0:00.00 sudo
 5069 root      20   0  165544   9704   8220 S   0.0  0.0   0:00.01 su
 5072 u19260    20   0   3

 6799 u19301    20   0   31176  10632   5900 S   0.0  0.0   0:00.03 echo_pid_a+
 6817 u19301    20   0    4628    784    720 S   0.0  0.0   0:00.00 sh
 6823 u19301    20   0  377016  85756  19748 S   0.0  0.1   0:03.11 jupyter-no+
 6829 root      20   0  122692   8004   7032 S   0.0  0.0   0:00.00 sudo
 6830 root      20   0  165544   9684   8200 S   0.0  0.0   0:00.01 su
 6832 u19302    20   0   31176  10604   5872 S   0.0  0.0   0:00.03 echo_pid_a+
 6857 u19302    20   0    4628    772    708 S   0.0  0.0   0:00.00 sh
 6868 u19302    20   0  376848  85380  19484 S   0.0  0.1   0:03.02 jupyter-no+
 6877 root      20   0  122692   7788   6812 S   0.0  0.0   0:00.01 sudo
 6878 root      20   0  165544   9544   8064 S   0.0  0.0   0:00.00 su
 6881 u19303    20   0   31176  10616   5884 S   0.0  0.0   0:00.03 echo_pid_a+
 6905 u19303    20   0    4628    860    796 S   0.0  0.0   0:00.00 sh
 6920 u19303    20   0  376192  85280  19664 S   0.0  0.1   0:03.62 jupyter-no+
 6928 root      20 

 7722 u19323    20   0   31176  10652   5916 S   0.0  0.0   0:00.03 echo_pid_a+
 7737 u19323    20   0    4628    776    712 S   0.0  0.0   0:00.00 sh
 7752 u19323    20   0  222372  78328  17524 S   0.0  0.1   0:02.46 jupyter-no+
 7756 root      20   0  122692   7744   6768 S   0.0  0.0   0:00.00 sudo
 7757 root      20   0  165544   9456   7976 S   0.0  0.0   0:00.01 su
 7760 u19324    20   0   31176  10624   5896 S   0.0  0.0   0:00.04 echo_pid_a+
 7783 u19324    20   0    4628    820    752 S   0.0  0.0   0:00.00 sh
 7792 u19324    20   0  222356  78376  17588 S   0.0  0.1   0:02.48 jupyter-no+
 7799 root      20   0  122692   8020   7044 S   0.0  0.0   0:00.01 sudo
 7800 root      20   0  165544   9632   8152 S   0.0  0.0   0:00.00 su
 7801 u19325    20   0   31176  10752   6020 S   0.0  0.0   0:00.03 echo_pid_a+
 7816 u19325    20   0    4628    780    712 S   0.0  0.0   0:00.00 sh
 7840 u19325    20   0  222320  78252  17460 S   0.0  0.1   0:02.46 jupyter-no+
 7842 root      20 

手元の端末では
```
$ top
```
とすると定期的に状況を更新してくれる

## 3. (参考) サーバ環境へ端末でアクセスする

## 3-1. (参考) Jupyter端末

* Jupyter上で動く端末を開くことも出来る. 自分のラップトップがLinuxでない人はこれを利用して, Linux固有のコマンドを実行することも出来る
* 画面左上Jupyter アイコン <img src="img/jupyter.png" width=60 /> をクリックし, そこからNew メニュー <img src="img/jupyter-new-menu.png" width=200 /> を開き, "Terminal"を選択すると, 以下のような端末画面が現れる.

<img src="img/jupyter-terminal.png" width=400 />

## 3-2. (参考) SSHでログイン

* サーバにネットワーク越しにアクセスして端末を開いたりコマンドを実行するのに使われる標準的な方法は, SSH (Secure Shell)というもので, 本演習のために必須ではないがあためしにやってみたいという人は, os00_ssh.ipynb を読んでやってみてください. 

* ただし以下の演習は Jupyter 上で実行すること. 理由:
 * 使用可能メモリ量を制限するためにJupyterに仕掛けを施しているため
 * Jupyter上の作業記録で提出内容の確認・評価を行うため

## 4. (重要) 本実験環境で使える物理メモリ量に関する注意

* 上記をやった人にはわかるとおり, 演習環境は64 GBの物理メモリを積んだクラウド環境上のサーバ
* 物理メモリを越えた領域を何度もアクセスして(メジャー)ページフォルト(スラッシング)を起こす実験をするにはこの物理メモリは大きすぎる
* それよりも何よりも, それをやってしまうと実験環境がまるごとメモリ不足によるスラッシング(事実上, 機能不全)に陥ってしまう
* 同じ環境を全学生が共有しているので, 他の人の実験もままならなくなる
* そこで, あえて個々のプロセスが使える物理メモリ量を, 搭載物理メモリよりも遥かに小さく制限して実験をする

* 方法はさておき, とりあえず「<font color="red">各学生のJupyter環境の物理メモリ量が256MBに制限</font>されている」と覚えておいてください

* 制限する方法に興味のある人は, このページ下部の, <a href="#cgroup_memory">cgroupプロセスが使えるメモリ量を制限する方法</a>を読んで下さい. これをやれば自分のラップトップでも同じことが出来ます.

## 5. メモリ割り当て

* まずは「仮想」メモリの割当ての実験
* 以下は A MB の仮想メモリを B 回 (合計 (A * B) MB), mallocを使って割り当てる
```
./alloc_1 A B
```
* mallocはOSのシステムコールではない. よって一般にはmallocをするたびにメモリ割り当てのためのシステムコール(Linuxであればsbrkかmmap)を呼び出すとは限らない. しかしこの実験のようにメモリをfreeすることもなくひたすら大きめのメモリを割り当て続けるプログラムにおいては, malloc = OSからのメモリ割り当て(sbrkかmmapを呼び出す)と考えても良い
* mallocは割り当てられた領域を初期化しない(触らない)ので, malloc = 仮想メモリ(のみ)の割当であり, 物理メモリの割当を伴わない
* 従ってmallocは物理メモリ量を越えるサイズであっても成功するし, 時間もほとんどかからない

In [5]:
//% file: alloc_1.c
//% cmd: gcc -O3 -Wall -Wextra -o alloc_1 alloc_1.c


#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void die(const char * s) {
  perror(s);
  exit(1);
}

/* ページサイズ */
const long page_sz = 1 << 12;     /* 4096 */

/* 秒単位で時刻を返す.
   初めて呼び出された時が時刻0 */
double cur_time() {
  static long t0 = 0;
  struct timeval tp[1];
  gettimeofday(tp, 0);
  long t = tp->tv_sec * 1000000 + tp->tv_usec;
  if (t0 == 0) t0 = t;
  return (t - t0) * 1.0e-6;
}

int main(int argc, char ** argv) {
  /* default 16MB x 64 = 1GB */
  long A = (argc > 1 ? atol(argv[1]) : 16);
  long B = (argc > 2 ? atol(argv[2]) : 64);
  long sz = A * 1024L * 1024L;
  cur_time();
  /* B回繰り返し */
  for (long i = 0; i < B; i++) {
    /* A MB割当て */
    char * a = malloc(sz);
    if (!a) die("malloc");
  }
  double t = cur_time();
  printf("%ld MB allocated in %e sec\n", A * B, t);
  return 0;
}


* 例えば以下は 1024 MB (1 GB) x 100 = 100 GB の malloc を行う.
* これが成功してしかも一瞬で終わるのはmallocしただけで実際に物理メモリは消費していないため

In [6]:
//% cmd: /usr/bin/time ./alloc_1 1024 100 2>&1

102400 MB allocated in 5.000000e-04 sec
0.00user 0.00system 0:00.00elapsed 100%CPU (0avgtext+0avgdata 1968maxresident)k
0inputs+0outputs (0major+161minor)pagefaults 0swaps


/usr/bin/time はコマンドの実行時間の他, プログラムが利用したメモリ量なども表示する.
例えば,

```
0.00user 0.00system 0:00.00elapsed 100%CPU (0avgtext+0avgdata 2036maxresident)k
0inputs+0outputs (0major+161minor)pagefaults 0swaps
```

は, maxresident (常駐メモリ量の最大 = 使用物理メモリ量の最大値) が 2036 kB = 約2 MB であったことを示している.  (0major+161minor)pagefaults は, メジャーページフォルトが0回, マイナーページフォルトが161回あったということ.

<font color="red">注: </font> Jupyter環境の不具合のせいで, 出力されるべき表示が出力されないことが, よくある. 何度か繰り返して表示されるまで繰り返してください m(_ _)

## <font color="blue">課題 5-1:</font> /usr/bin/time の出力

上記で実行した /usr/bin/time ./alloc_1 1024 100 の出力を読み取り,

* (プログラムが表示する)実行時間
* 使用物理メモリ量(の最大値)
* マイナーページフォルトの数
* メジャーページフォルトの数

を, 以下のセルを修正して答えよ. 後者3つの数字が上記のプログラムの動作に照らして理にかなっているものかどうかを考察せよ.

<font color="blue">このセルを修正して解答を書け. Modify this cell and answer the questions.</font>

* (プログラムが表示する)実行時間 = 5.000000e-04 秒
* 使用物理メモリ量(の最大値)     = 1968 KB
* マイナーページフォルトの数     = 161 回
* メジャーページフォルトの数     = 0 回
* 考察 : 
使用物理メモリ量は100GBより十分小さい。それは仮想記憶が使われ、物理メモリは確保されていないからである。
仮想記憶に確保された100GBはアクセスされないが、ソースコード中で時間を図ったり、変数A,Bなどが使用されているのでそこでマイナーページフォルトが使われている。
最後に物理メモリは2MB程しか使用されていないので2次記憶にpage-outする必要がなく、メジャーフォルトは0回である。

## <font color="blue">課題 5-2:</font> 物理メモリを割り当てるか否かの違い

* 上記のプログラムを変更し, mallocした直後に割り当てられた領域の各ページ(4KB)に一度ずつ書き込みを行うよう変更せよ. 
* そのもとで, A = 16, B = 10, 20, 30 と変えて実行時間を観察せよ.
* その上でその下の問いに答えよ.

In [22]:
//% file: alloc_2.c
//% cmd: gcc -O3 -Wall -Wextra -o alloc_2 alloc_2.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void die(const char * s) {
  perror(s);
  exit(1);
}

/* ページサイズ */
const long page_sz = 1 << 12;     /* 4096 */

/* 秒単位で時刻を返す.
   初めて呼び出された時が時刻0 */
double cur_time() {
  static long t0 = 0;
  struct timeval tp[1];
  gettimeofday(tp, 0);
  long t = tp->tv_sec * 1000000 + tp->tv_usec;
  if (t0 == 0) t0 = t;
  return (t - t0) * 1.0e-6;
}

int main(int argc, char ** argv) {
  /* default 16MB x 64 = 1GB */
  long A = (argc > 1 ? atol(argv[1]) : 16);
  long B = (argc > 2 ? atol(argv[2]) : 64);
  long sz = A * 1024L * 1024L;
  cur_time();
  /* B回繰り返し */
  for (long i = 0; i < B; i++) {
    /* A MB割当て */
    char * a = malloc(sz);
    if (!a) die("malloc");
    /* 以下で各ページに一度ずつアクセス(読み書き)せよ */
    for (long j=0; j<sz; j+=4096L){
        a[j]='a';
    }
  }
  double t = cur_time();
  printf("%ld MB allocated in %e sec\n", A * B, t);
  return 0;
}


In [29]:
//%cmd: /usr/bin/time ./alloc_1 16 10 2>&1

160 MB allocated in 7.400000e-05 sec
0.00user 0.00system 0:00.00elapsed ?%CPU (0avgtext+0avgdata 1640maxresident)k
24inputs+0outputs (0major+74minor)pagefaults 0swaps


In [30]:
//%cmd: /usr/bin/time ./alloc_2 16 10 2>&1

160 MB allocated in 7.845200e-02 sec
0.02user 0.05system 0:00.08elapsed 98%CPU (0avgtext+0avgdata 165456maxresident)k
0inputs+0outputs (0major+41025minor)pagefaults 0swaps


In [31]:
//%cmd: /usr/bin/time ./alloc_1 16 20 2>&1

320 MB allocated in 1.420000e-04 sec
0.00user 0.00system 0:00.00elapsed ?%CPU (0avgtext+0avgdata 1644maxresident)k
0inputs+0outputs (0major+81minor)pagefaults 0swaps


In [32]:
//%cmd: /usr/bin/time ./alloc_2 16 20 2>&1

320 MB allocated in 4.613990e-01 sec
0.02user 0.21system 0:00.47elapsed 50%CPU (0avgtext+0avgdata 248504maxresident)k
56inputs+0outputs (7major+81986minor)pagefaults 0swaps


In [33]:
//%cmd: /usr/bin/time ./alloc_1 16 30 2>&1

480 MB allocated in 1.970000e-04 sec
0.00user 0.00system 0:00.00elapsed 0%CPU (0avgtext+0avgdata 1796maxresident)k
24inputs+0outputs (0major+93minor)pagefaults 0swaps


In [34]:
//%cmd: /usr/bin/time ./alloc_2 16 30 2>&1

480 MB allocated in 1.216241e+00 sec
0.02user 0.37system 0:01.23elapsed 32%CPU (0avgtext+0avgdata 251456maxresident)k
56inputs+0outputs (5major+122946minor)pagefaults 0swaps


* ./alloc_1 16 10 と ./alloc_2 16 10 の実行時間の違いは何によるものか?
* ./alloc_2 16 10 と ./alloc_2 16 20 の実行時間を比べると, 後者は前者の2倍では全くすまない. それは何によるものか?

<font color="blue">このセルを修正して解答を書け. Modify this cell and answer the questions.</font>

* ./alloc_1 16 10 と ./alloc_2 16 10 の実行時間の違いは何によるものか?
 * 答:前者の方がminor faultが非常に多く、maxresidentの値からもわかるように実際に物理メモリが確保され、minor faultを起こしていることがわかる。それにより後者の方が実行時間が10^3のオーダーで遅くなっている。
* ./alloc_2 16 10 と ./alloc_2 16 20 の実行時間を比べると, 後者は前者の2倍では全くすまない. それは何によるものか?
 * 答:後者は倍の領域を確保しているにもかかわらず、物理メモリは256MBしかないため確保できなかった部分を2次記憶領域に書き出している。それは後者のmaxresidentが前者の2倍となっていないことからもわかる。それによりmajor faultを起こし実行時間が2倍以上かかってしまう。

## 6. ページフォルト数の計測

* 要求時ページングと2次記憶からのページイン・アウトが起きていることを実際に観測してみよう

* 以下は

```
$ ./record_page_faults_1 A
```

とすると以下を実行する

1. A MB のメモリを(mallocで)割当て
1. 「その各ページに一度ずつ読み書きする」を3回繰り返す

それをやりながら時々(256ページ毎に一回), 以下を記録する

1. 触ったページの延べ数 x 4096 (x)
1. 経過時間(malloc呼び出し直前からの時間)(t)

最後に上記を出力する. 各行が 
```
x t
```
の形式

In [35]:
//% file: record_page_faults_1.c
//% cmd: gcc -O3 -Wall -Wextra -o record_page_faults_1 record_page_faults_1.c


#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/time.h>
#include <sys/resource.h>

void die(const char * s) {
  perror(s);
  exit(1);
}

/* ページサイズ */
const long page_sz = 1 << 12;     /* 4096 */

/* 記録する頻度 (1MB分のページに触る毎に記録) */
const long record_gran = 1 << 20; /* 1 MB */

/* 秒単位で時刻を返す.
   初めて呼び出された時が時刻0 */
double cur_time() {
  static long t0 = 0;
  struct timeval tp[1];
  gettimeofday(tp, 0);
  long t = tp->tv_sec * 1000000 + tp->tv_usec;
  if (t0 == 0) t0 = t;
  return (t - t0) * 1.0e-6;
}

/* 記録の1エントリ */
typedef struct {
  long x;                       /* x軸の値 (ページ数, バイト数, etc.) */
  double t;                     /* 経過時間 */
} record_t;

/* 記録を貯めるバッファ */
typedef struct {
  long capacity;
  long n;
  record_t * b;
} records_t;

/* 記録を貯めるバッファを作る */
records_t * mk_records(long capacity) {
  record_t * b
    = (record_t *)malloc(sizeof(record_t) * capacity);
  memset(b, 0, sizeof(record_t) * capacity);
  records_t * bs = (records_t *)malloc(sizeof(records_t));
  bs->capacity = capacity;
  bs->n = 0;
  bs->b = b;
  return bs;
}

/* 時刻やページフォルトの回数を記録 */
void record_time_and_faults(records_t * bs, long x) {
  double t = cur_time();
  long n = bs->n;
  assert(n < bs->capacity);
  bs->b[n].x = x;
  bs->b[n].t = t;
  bs->n = n + 1;
}

/* 記録された時刻とページフォルト数をファイルに保存 */
void save_records(records_t * bs, const char * filename) {
  FILE * wp = fopen(filename, "w");
  if (!wp) die("fopen");
  long n = bs->n;
  for (long i = 0; i < n; i++) {
    fprintf(wp, "%ld %f\n", 
            bs->b[i].x, bs->b[i].t);
  }
  fclose(wp);
}

/* A MB 割り当ててそれを B 回スキャンする.
   記録を log に書く */
int main(int argc, char ** argv) {
  long A = (argc > 1 ? atol(argv[1]) : 128);
  long B = (argc > 2 ? atol(argv[2]) : 3);
  char * log = (argc > 3 ? argv[3] : "rpf.log");
  long sz = A * 1024L * 1024L;
  long n_recs = ((sz + record_gran - 1) / record_gran) * B;
  records_t * bs = mk_records(n_recs);
  cur_time();
  /* A MB割当て */
  char * a = malloc(sz);
  if (!a) die("malloc");
  /* B回, 頭から終わりまで全ページに触る */
  long k = 0;
  for (long i = 0; i < B; i++) {
    for (long j = 0; j < sz; j += page_sz) {
      if (k % record_gran == 0) record_time_and_faults(bs, k);
      a[j]++;
      k += page_sz;
    }
  }
  save_records(bs, log);
  return 0;
}


以下は 150MB (< 256MB)の領域の各ページを3回触る

In [36]:
//% cmd: /usr/bin/time ./record_page_faults_1 150 3 time150x3.log 2>&1

0.01user 0.05system 0:00.07elapsed 98%CPU (0avgtext+0avgdata 155224maxresident)k
0inputs+24outputs (0major+38466minor)pagefaults 0swaps


* 別のJupyterノートブック os05_mem_visualize.ipynb を開くとこれを可視化出来るようになっている
* 左上のJupyterアイコン <img src="img/jupyter.png" width=60 /> をクリックし, os05_mem_visualize.ipynb を開いてあとは指示に従うこと

以下は 300MB (> 256MB)の領域の各ページを3回触る

In [37]:
//% cmd: /usr/bin/time ./record_page_faults_1 300 3 time300x3.log 2>&1

0.11user 0.91system 0:09.57elapsed 10%CPU (0avgtext+0avgdata 250584maxresident)k
1209144inputs+40outputs (19227major+211241minor)pagefaults 0swaps


以下は 450MB (> 256MB)の領域の各ページを3回触る

In [38]:
//% cmd: /usr/bin/time ./record_page_faults_1 450 3 time450x3.log 2>&1

0.20user 1.44system 0:15.14elapsed 10%CPU (0avgtext+0avgdata 253036maxresident)k
1847032inputs+56outputs (29030major+316664minor)pagefaults 0swaps


## <font color="blue">課題 5-3:</font> ページフォルト数の表示

* システムコールgetrusage のマニュアルページを見よ(Linuxの人はmanコマンドで. そうでない人はJupyter Terminalやgoogleなどで)
* プログラム(records_t, record_time_and_faults, save_recordsなど)を変更して, 経過時間と合わせて, マイナーページフォルト数, メジャーページフォルト数を記録・保存するようにせよ
* 結果を予想した上で, os05_mem_visualize.ipynb で可視化せよ. 可視化したらどうしてそうなるか, その説明をせよ

In [45]:
//% file: record_page_faults_2.c
//% cmd: gcc -O3 -Wall -Wextra -o record_page_faults_2 record_page_faults_2.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/time.h>
#include <sys/resource.h>

void die(const char * s) {
  perror(s);
  exit(1);
}

/* ページサイズ */
const long page_sz = 1 << 12;     /* 4096 */

/* 記録する頻度 (1MB分のページに触る毎に記録) */
const long record_gran = 1 << 20; /* 1 MB */

/* 秒単位で時刻を返す.
   初めて呼び出された時が時刻0 */
double cur_time() {
  static long t0 = 0;
  struct timeval tp[1];
  gettimeofday(tp, 0);
  long t = tp->tv_sec * 1000000 + tp->tv_usec;
  if (t0 == 0) t0 = t;
  return (t - t0) * 1.0e-6;
}

/* 記録の1エントリ */
typedef struct {
  long x;                       /* x軸の値 (ページ数, バイト数, etc.) */
  double t;                     /* 経過時間 */
  struct rusage usage; //changed
} record_t;

/* 記録を貯めるバッファ */
typedef struct {
  long capacity;
  long n;
  record_t * b;
} records_t;

/* 記録を貯めるバッファを作る */
records_t * mk_records(long capacity) {
  record_t * b
    = (record_t *)malloc(sizeof(record_t) * capacity);
  memset(b, 0, sizeof(record_t) * capacity);
  records_t * bs = (records_t *)malloc(sizeof(records_t));
  bs->capacity = capacity;
  bs->n = 0;
  bs->b = b;
  return bs;
}

/* 時刻やページフォルトの回数を記録 */
void record_time_and_faults(records_t * bs, long x) {
  double t = cur_time();
  long n = bs->n;
  assert(n < bs->capacity);
  bs->b[n].x = x;
  bs->b[n].t = t;
  if(getrusage(RUSAGE_SELF, &bs->b[n].usage) < 0) die("getrusage.\n");//changed
  bs->n = n + 1;
}

/* 記録された時刻とページフォルト数をファイルに保存 */
void save_records(records_t * bs, const char * filename) {
  FILE * wp = fopen(filename, "w");
  if (!wp) die("fopen");
  long n = bs->n;
  for (long i = 0; i < n; i++) {
    //changed
    fprintf(wp, "%ld %f %ld %ld\n", 
            bs->b[i].x, bs->b[i].t, bs->b[i].usage.ru_minflt, bs->b[i].usage.ru_majflt);
  }
  fclose(wp);
}

/* A MB 割り当ててそれを B 回スキャンする.
   記録を log に書く */
int main(int argc, char ** argv) {
  long A = (argc > 1 ? atol(argv[1]) : 128);
  long B = (argc > 2 ? atol(argv[2]) : 3);
  char * log = (argc > 3 ? argv[3] : "rpf.log");
  long sz = A * 1024L * 1024L;
  long n_recs = ((sz + record_gran - 1) / record_gran) * B;
  records_t * bs = mk_records(n_recs);
  cur_time();
  /* A MB割当て */
  char * a = malloc(sz);
  if (!a) die("malloc");
  /* B回, 頭から終わりまで全ページに触る */
  long k = 0;
  for (long i = 0; i < B; i++) {
    for (long j = 0; j < sz; j += page_sz) {
      if (k % record_gran == 0) record_time_and_faults(bs, k);
      a[j]++;
      k += page_sz;
    }
  }
  save_records(bs, log);
  return 0;
}

/*マイナーフォルトの解釈：物理領域より小さい領域を確保する場合は、
一度マイナーフォルト起こしてメモリ内に格納すれば、2回目からはマイナーフォルトを起こすことなく、
アクセスすることができる。一方で、確保する領域が物理領域を超える場合は一度アクセスした後、
2次記憶にページアウトされ再びアクセスするときはページインしアクセスするので毎回マイナーフォルトを
起こす。*/

/*メジャーフォルトの解釈：物理領域より小さい場合はメモリで事足りるのでメジャーフォルトは起きない。
しかし、大きい場合はページイン・ページアウトして2次記憶も用いる。するとメジャーフォルトが起き、
メモリ内のページが2次記憶に格納されるので2回目のアクセスからはメジャーフォルトをおこす。*/


150MB (< 256MB)の領域の各ページを3回触る

In [46]:
//% cmd: /usr/bin/time ./record_page_faults_2 150 3 pf150x3.log 2>&1

300MB (> 256MB)の領域の各ページを3回触る

In [47]:
//% cmd: /usr/bin/time ./record_page_faults_2 300 3 pf300x3.log 2>&1

0.11user 0.96system 0:10.34elapsed 10%CPU (0avgtext+0avgdata 246144maxresident)k
1240968inputs+56outputs (20423major+210142minor)pagefaults 0swaps


450MB (> 256MB)の領域の各ページを3回触る

In [48]:
//% cmd: /usr/bin/time ./record_page_faults_2 450 3 pf450x3.log 2>&1

0.12user 1.60system 0:16.80elapsed 10%CPU (0avgtext+0avgdata 250656maxresident)k
1848032inputs+88outputs (29284major+316537minor)pagefaults 0swaps


## <font color="blue">課題 5-4:</font> メジャーフォルト「率」の表示

* 上記に加え, 各記録区間のメジャーフォルトの「率」を保存・表示せよ
* 記録区間とは, 2回の隣り合う記録の間ということで, record_time_and_faultsが$(i-1)$回目に呼ばれた時点と, $i$回目に呼ばれた時点の間が, $i$番目の区間ということである(0番目の区間は存在せず, 便宜上メジャーフォルト率は0とする).
* ある区間におけるメジャーフォルト率は,

 その区間におけるメジャーフォルト数 / その区間において触ったページ数

とする. 触ったページ数は配列 a の中のページ数だけを数えれば良い. 要するに, a[j]++ を実行した回数に過ぎない

* 結果を予想してから実行, そして可視化して結果を確認・解釈せよ.

* その上でその下の問いに答えよ.

In [49]:
//% file: record_page_faults_3.c
//% cmd: gcc -O3 -Wall -Wextra -o record_page_faults_3 record_page_faults_3.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/time.h>
#include <sys/resource.h>

void die(const char * s) {
  perror(s);
  exit(1);
}

/* ページサイズ */
const long page_sz = 1 << 12;     /* 4096 */

/* 記録する頻度 (1MB分のページに触る毎に記録) */
const long record_gran = 1 << 20; /* 1 MB */

/* 秒単位で時刻を返す.
   初めて呼び出された時が時刻0 */
double cur_time() {
  static long t0 = 0;
  struct timeval tp[1];
  gettimeofday(tp, 0);
  long t = tp->tv_sec * 1000000 + tp->tv_usec;
  if (t0 == 0) t0 = t;
  return (t - t0) * 1.0e-6;
}

/* 記録の1エントリ */
typedef struct {
  long x;                       /* x軸の値 (ページ数, バイト数, etc.) */
  double t;                     /* 経過時間 */
  struct rusage usage; //changed
} record_t;

/* 記録を貯めるバッファ */
typedef struct {
  long capacity;
  long n;
  record_t * b;
} records_t;

/* 記録を貯めるバッファを作る */
records_t * mk_records(long capacity) {
  record_t * b
    = (record_t *)malloc(sizeof(record_t) * capacity);
  memset(b, 0, sizeof(record_t) * capacity);
  records_t * bs = (records_t *)malloc(sizeof(records_t));
  bs->capacity = capacity;
  bs->n = 0;
  bs->b = b;
  return bs;
}

/* 時刻やページフォルトの回数を記録 */
void record_time_and_faults(records_t * bs, long x) {
  double t = cur_time();
  long n = bs->n;
  assert(n < bs->capacity);
  bs->b[n].x = x;
  bs->b[n].t = t;
  if(getrusage(RUSAGE_SELF, &bs->b[n].usage) < 0) die("getrusage.\n");//changed
  bs->n = n + 1;
}

/* 記録された時刻とページフォルト数をファイルに保存 */
void save_records(records_t * bs, const char * filename) {
  FILE * wp = fopen(filename, "w");
  if (!wp) die("fopen");
  long n = bs->n;
  for (long i = 0; i < n; i++) {
    /* -------------------- 
       マイナーフォルト数, メジャーフォルト数, 直近区間における
       メジャーページフォルト「率」を(列を追加して)保存せよ 
       -------------------- */
    //changed
    float ratio=0;
    if(i!=0){
        long pagenum=(bs->b[i].x - bs->b[i-1].x)/page_sz;
        ratio=(float)(bs->b[i].usage.ru_majflt - bs->b[i-1].usage.ru_majflt) / pagenum;
    }
    fprintf(wp, "%ld %f %ld %ld %f\n", 
            bs->b[i].x, bs->b[i].t, bs->b[i].usage.ru_minflt, bs->b[i].usage.ru_majflt,
            ratio);
  }
  fclose(wp);
}

/* A MB 割り当ててそれを B 回スキャンする.
   記録を log に書く */
int main(int argc, char ** argv) {
  long A = (argc > 1 ? atol(argv[1]) : 128);
  long B = (argc > 2 ? atol(argv[2]) : 3);
  char * log = (argc > 3 ? argv[3] : "rpf.log");
  long sz = A * 1024L * 1024L;
  long n_recs = ((sz + record_gran - 1) / record_gran) * B;
  records_t * bs = mk_records(n_recs);
  cur_time();
  /* A MB割当て */
  char * a = malloc(sz);
  if (!a) die("malloc");
  /* B回, 頭から終わりまで全ページに触る */
  long k = 0;
  for (long i = 0; i < B; i++) {
    for (long j = 0; j < sz; j += page_sz) {
      if (k % record_gran == 0) record_time_and_faults(bs, k);
      a[j]++;
      k += page_sz;
    }
  }
  save_records(bs, log);
  return 0;
}


150MB (< 256MB)の領域の各ページを3回触る

In [50]:
//% cmd: /usr/bin/time ./record_page_faults_3 150 3 pfrate150x3.log 2>&1

300MB (> 256MB)の領域の各ページを3回触る

In [51]:
//% cmd: /usr/bin/time ./record_page_faults_3 300 3 pfrate300x3.log 2>&1

0.15user 0.91system 0:09.68elapsed 11%CPU (0avgtext+0avgdata 250212maxresident)k
1215456inputs+72outputs (19295major+211248minor)pagefaults 0swaps


450MB (> 256MB)の領域の各ページを3回触る

In [52]:
//% cmd: /usr/bin/time ./record_page_faults_3 450 3 pfrate450x3.log 2>&1

0.18user 1.38system 0:15.04elapsed 10%CPU (0avgtext+0avgdata 250396maxresident)k
1849000inputs+112outputs (28987major+316837minor)pagefaults 0swaps


<font color="blue">このセルを修正して解答を書け. Modify this cell and answer the questions.</font>

* ページ置換が厳密なLRUで行われ, かつページインは要求されたページに対してのみ行われるとすると, 配列のサイズが物理メモリ量を上回る場合のメジャーページフォルト率はいくらになるはずか?
 * 答:メジャーページフォルトは各要求に対して1度のみ起こるのでメジャーページフォルト率は1になる。

* 実際に観測された, 配列のサイズが物理メモリ量を上回る場合のメジャーページフォルト率はいくつだったか
 * 答:0.125程度であった。

* 両者の違いはどのような理由で生ずるか考えられる理由を書け
 * 答:実際には1度のページイン要求に対して、近くのページもろとも一括でページインしているので要求されるページの数よりページフォルトが小さい値になっていると考えれる。

## 7. (参考) Linux control groups を用いた使用メモリ量の制限

* この実験では Linux control groups (cgroups) という機能を用いて, 特定プロセス(とその子孫)が利用可能な物理メモリの量を人工的に制限していた
* さもないと, 物理メモリ料を上回るメモリを扱う実験では非常に大きなメモリを使うことになる(ので実験にとてつもない時間がかかるようになる)上, そんなことをしたら環境事態が不安定になってしまうのでこのようにしている
* 参考までにそのやり方を説明する. 自分のPCがラップトップという人は以下の手順で同じことが出来る

* 以下の手順はすべて, 自分のPCでcgroupsを使うための手順で, コマンドもすべて自分の(Linux) PCで実行するものなのでお間違いなきよう

## 7-1. スワップ領域が有効であることを確かめる

```
$ free
              total        used        free      shared  buff/cache   available
Mem:       32562996     5504400    11225244      714392    15833352    25862016
Swap:       2097148        7936     2089212

```

で Swap の total が 0でなければOK. もしここが0だとページング領域がない(か, 無効化されている)ということで, 有効にする(<a href="#enable_swap">その方法</a>).

## 7-2. cgroupsのバージョンを確かめる

cgroupsにはバージョン1と2があり, 両者に互換性はないのでまずどちらであるかを判別する.

```
$ ls /sys/fs/cgroup
```

* 結果がこんな感じ(memory というサブフォルダがある)であれば ver. 1 

```
deneb:~$ ls /sys/fs/cgroup
blkio/	  cpu,cpuacct/	freezer/  perf_event/  systemd/
cpu@	  cpuset/	hugetlb/  pids/	  unified/
cpuacct@  devices/	memory/   rdma/
```

* こんな感じ(cgroup.procs というファイルがある)であれば ver. 2

```
$ ls /sys/fs/cgroup
cgroup.controllers	cgroup.subtree_control
cgroup.max.depth	cgroup.threads		system.slice/
cgroup.max.descendants	cpuset.cpus.effective	user.slice/
cgroup.procs		cpuset.mems.effective
cgroup.stat		init.scope/
```

## 7-3. ver. 1 の場合

1. /sys/fs/cgroup/memory 下に適当なサブフォルダを作る(例として my_group とする)
1. /sys/fs/cgroup/memory/my_group 下に多数のファイルがひとりでに出来る
1.  /sys/fs/cgroup/memory/memory.limit_in_bytes に数値(L)を書き込む
1.  /sys/fs/cgroup/memory/my_group/tasks に配下におきたいプロセスのID (pとする)を書き込む

これによって, p およびその子プロセスで使えるメモリ量がLに制限される. 

例

```
$ sudo mkdir -p /sys/fs/cgroup/memory/my_group
$ echo 256M | sudo tee /sys/fs/cgroup/memory/my_group/memory.limit_in_bytes 
$ echo $$ | sudo tee /sys/fs/cgroup/memory/my_group/tasks
... 以降このシェルとそこから実行されるコマンドに256MBの制限が適用される ...
```

## 7-4. ver. 2 の場合

1. /sys/fs/cgroup 下に適当なサブフォルダを作る(例として my_group とする)
1. /sys/fs/cgroup/my_group 下に多数のファイルがひとりでに出来る
1. /sys/fs/cgroup/cgroup.subtree_control に "+memory" と書き込む
1. /sys/fs/cgroup/my_group/memory.high に数値(L)を書き込む
1.  /sys/fs/cgroup/my_group/cgroup.procs に配下におきたいプロセスのID (pとする)を書き込む

これによって, p およびその子プロセスで使えるメモリ量がLに制限される

例

```
$ sudo mkdir -p /sys/fs/cgroup/my_group
$ echo +memory | sudo tee /sys/fs/cgroup/cgroup.subtree_control 
$ echo 256M | sudo tee /sys/fs/cgroup/my_group/memory.high
$ echo $$ | sudo tee /sys/fs/cgroup/my_group/cgroup.procs
... 以降このシェルとそこから実行されるコマンドに256MBの制限が適用される ...
```

## 7-5. 両方に対応したシェルスクリプト

* <a href="https://taulec.zapto.org/~tau/problems/os/download/mem_limit.sh">
このシェルスクリプト</a>を保存して実行すると, 子供のシェル(bash)が開き, そのシェルおよびそこから起動されるコマンドに, 256MB の制限が適用される.

* そのシェルから抜ければ元(制限のない状態)に戻る

* 適宜修正して使用されたい(例えば zsh が好きな人は最後の行を zsh に変える)

<a name="enable_swap"> </a>
## 7-6. swap領域を有効化する方法

* <a href="https://linuxize.com/post/create-a-linux-swap-file/">
このページ
</a> を参照

* 同じことを<a href="https://taulec.zapto.org/~tau/problems/os/download/mk_swap_and_on.sh">シェルスクリプト</a>にしたもの. 適宜変更して利用されたい